In [ ]:
import importlib

import dataset, metrics, plotting, config
from models import cgan_model
import numpy as np
import random

importlib.reload(dataset)
importlib.reload(metrics)
importlib.reload(plotting)
importlib.reload(config)
importlib.reload(cgan_model)

In [ ]:
import os

dataset_config = config.DatasetConfig(scenario="heteroscedastic", n_instance=1000)

assert(dataset_config.scenario == "linear" 
       or dataset_config.scenario == "sinus"
       or dataset_config.scenario == "heteroscedastic"
       or dataset_config.scenario == "exp"
       or dataset_config.scenario == "multi-modal"
      )
fig_dir = f"../figures/{dataset_config.scenario}"

try:
    os.mkdir(fig_dir)
    print(f"Directory {fig_dir} created ") 
except FileExistsError:
    print(f"Directory {fig_dir} already exists replacing files in this notebook")

In [ ]:
exp_config = config.Config(
    model=config.ModelConfig(activation="elu", lr_gen=0.0001, dec_gen=0, lr_disc=0.001, optim_gen="Adam", 
                             optim_disc="Adam", z_input_size=1),
    training=config.TrainingConfig(n_epochs=2000, batch_size=100, n_samples=100),
    dataset=dataset_config,
    run=config.RunConfig(save_fig=1)
)

In [ ]:
# Set random seed
np.random.seed(exp_config.model.random_seed)
random.seed(exp_config.model.random_seed)

from tensorflow import set_random_seed
set_random_seed(exp_config.model.random_seed)

In [ ]:
X_train, y_train, X_test, y_test, X_valid, y_valid = dataset.get_dataset(exp_config.dataset.n_instance, 
                                                                         exp_config.dataset.scenario)

In [ ]:
plotting.plot_dataset(X_train, X_test, X_valid, y_train, y_test, y_valid, exp_config, fig_dir)

## Linear Regression 

In [ ]:
from sklearn import linear_model

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

ypred_linear_test = regr.predict(X_test)
ypred_linear_train = regr.predict(X_train)

## Gaussian Process 

In [ ]:
import GPy
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF

noise = 1
length = 1

run_hyperopt_search = True

kernel = GPy.kern.RBF(input_dim=1, variance=noise, lengthscale=length)
gpr = GPy.models.GPRegression(X_train, y_train, kernel)
if run_hyperopt_search:
    gpr.optimize(messages=True) 

In [ ]:
ypred_gp_test, cov_test = gpr.predict(X_test)

## GAN 

In [ ]:
cgan = cgan_model.CGAN(exp_config)
d_loss_err, d_loss_true, d_loss_fake, g_loss_err, g_pred, g_true = cgan.train(X_train, y_train, 
                                                                              epochs=exp_config.training.n_epochs,
                                                                              batch_size=exp_config.training.batch_size)

ypred_gan_test = cgan.predict(X_test)

In [ ]:
plotting.plots(d_loss_err, d_loss_true, d_loss_fake, g_loss_err, g_pred, g_true, fig_dir, exp_config.run.save_fig)

In [ ]:
ypred_mean_gan_test, ypred_median_gan_test, ypred_gan_sample_test = cgan.sample(X_test, exp_config.training.n_samples)

# Plots

## Mean Predictions 

In [ ]:
plotting.plot_ypred_joint(X_test, y_test, ypred_linear_test, ypred_mean_gan_test, ypred_gp_test, 
                          "Linear-vs-GAN-vs-GP Mean Test Predictions", fig_dir=fig_dir, prefix="all_mean", 
                          save_fig=exp_config.run.save_fig)

In [ ]:
# Single GAN sample
plotting.plot_ypred_joint(X_test, y_test, ypred_linear_test, ypred_gan_test, ypred_gp_test, 
                          "Linear-vs-GAN-vs-GP Test Single GAN sample", fig_dir=fig_dir, prefix="gan_single_sample", 
                          save_fig=exp_config.run.save_fig)

## Density 

In [ ]:
sample_size_contours = 2

plot_contours = False

if plot_contours:
    # Sample from GP P(Y|X)
    ypred_gp_sample_test = np.random.normal(ypred_gp_test, np.sqrt(cov_test))
    for i in range(1, sample_size_contours):
        ypred_gp_sample_test = np.vstack([ypred_gp_sample_test, np.random.normal(ypred_gp_test, np.sqrt(cov_test))])
    
    # Sample from GAN
    _, _, ypred_gan_sample_test = cgan.sample(X_test, sample_size_contours)
    
    plotting.plot_density_cont(X_test, y_test, title="True contours", fig_dir=fig_dir, prefix="true_sample", 
                               save_fig=exp_config.run.save_fig)
    plotting.plot_density_cont(np.tile(X_test, plotting_sample_size), ypred_gp_sample_test, title="GP contours", 
                               fig_dir=fig_dir, prefix="gp_sample", save_fig=exp_config.run.save_fig)
    plotting.plot_density_cont(np.tile(X_test, plotting_sample_size), ypred_gan_sample_test, title="GAN contours", 
                               fig_dir=fig_dir, prefix="gan_sample", save_fig=exp_config.run.save_fig)

In [ ]:
plotting.plot_densities_joint(y_test, ypred_linear_test, ypred_mean_gan_test, ypred_gp_test, 
                              "Marginalized P(y) density", fig_dir=fig_dir, prefix="all_marginalized", 
                              save_fig=exp_config.run.save_fig)

## Display predictions with more samples 

In [ ]:
plotting_sample_size = 2 # inscrease sample_size to have gaussian densities for GP

In [ ]:
_, _, X_test_sample, y_test_sample, _, _ = dataset.get_dataset(exp_config.dataset.n_instance * 5, 
                                                               exp_config.dataset.scenario)

In [ ]:
ypred_mean_gan_full_test, ypred_median_gan_full_test, ypred_gan_sample_full_test = cgan.sample(X_test_sample, 
                                                                                               plotting_sample_size)

In [ ]:
ypred_gp_test_full, cov_test_full = gpr.predict(X_test_sample)

ypred_gp_sample_full_test = np.random.normal(ypred_gp_test_full[0], np.sqrt(cov_test_full[0]))
for i in range(1, len(X_test_sample)):
    ypred_gp_sample_full_test = np.vstack([ypred_gp_sample_full_test, 
                                           np.random.normal(ypred_gp_test_full[i], np.sqrt(cov_test_full[i]))])

In [ ]:
if exp_config.dataset.scenario == "linear":
    ylim=[-10, 15]
elif exp_config.dataset.scenario == "sinus" or exp_config.dataset.scenario == "heteroscedastic":
    ylim=[-4, 4]
elif exp_config.dataset.scenario == "exp":
    ylim=[-4, 9]
elif exp_config.dataset.scenario == "multi-modal":
    ylim=[-0.1, 1.1]
else:
    raise Exception("Y limits to be specified")
    
plotting.plot_ypred_joint(X_test_sample, y_test_sample, None, None, None, "True Sample", alpha=0.15, is_sample=True, 
                          fig_dir=fig_dir, prefix="true_more_sample", save_fig=exp_config.run.save_fig, ylim=ylim)

In [ ]:
plotting.plot_ypred_joint(X_test_sample, None, None, ypred_gan_sample_full_test, None, 
                          "GAN Test Predictions", alpha=0.25, is_sample=True, fig_dir=fig_dir, 
                          prefix="gan_more_sample", save_fig=exp_config.run.save_fig, ylim=ylim)

In [ ]:
plotting.plot_ypred_joint(X_test_sample, None, None, None, ypred_gp_sample_full_test, "GP Test Predictions", 
                          alpha=0.15, is_sample=False, fig_dir=fig_dir, prefix="gp_more_sample", 
                          save_fig=exp_config.run.save_fig, ylim=ylim)

## P(y|x) 

In [ ]:
if exp_config.dataset.scenario == "sinus":
    bins = [-1, 0, 1]
    ylim = [[-2.5,2.5], [-2.5,2.5], [-2.5,2.5]]
elif exp_config.dataset.scenario == "linear":
    bins = [4, 0, 10] 
    ylim = [[-5,17.5], [-5,17.5], [5,22]]
elif exp_config.dataset.scenario == "exp":
    bins = [-1, 0, 1] 
    ylim = [[-5,17.5], [-5,17.5], [5,22]]
elif exp_config.dataset.scenario == "multi-modal":
    bins = [0.4, 0.45, 0.2]
    ylim = [[0,1.1], [0,1], [0,1.1]]
elif exp_config.dataset.scenario == "heteroscedastic":
    bins = [-2,0,2]
    ylim = [[-3.2,2], [-2,2], [-2,5.5]]
else:
    raise Exception("Bins to be specified")

ind = 0
delta = 0.05

for x in bins:
    idx = np.logical_and(X_test_sample < x + delta, X_test_sample > x - delta)
    ypred_gan_sample_ = ypred_gan_sample_full_test[:, 0].reshape(-1,1)
    ypred_gp_sample_ = ypred_gp_sample_full_test[:, 0].reshape(-1,1)
    plotting.plot_densities_joint(y_test_sample[idx], None, ypred_gan_sample_[idx], ypred_gp_sample_[idx], 
                                  f"P(y|x) density at x={x}", at_x=True, fig_dir=fig_dir, prefix=f"all_at_x_{x}", 
                                  save_fig=exp_config.run.save_fig, ylim=ylim[ind])
    ind = ind + 1

# Metrics

In [ ]:
n_eval_runs = 10

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mse_gan_= []
mae_gan_ = []
for i in range(n_eval_runs):
    ypred_mean_gan_test_, ypred_median_gan_test_, _ = cgan.sample(X_test, exp_config.training.n_samples)
    mae_gan_.append(mean_absolute_error(y_test, ypred_median_gan_test_))
    mse_gan_.append(mean_squared_error(y_test, ypred_mean_gan_test_))

In [ ]:
gan_mae_mean = np.mean(np.asarray(mae_gan_))
gan_mae_std = np.std(np.asarray(mae_gan_))

print(f"Linear MAE test: {mean_absolute_error(ypred_linear_test, y_test)}")
print(f"GP MAE test: {mean_absolute_error(ypred_gp_test, y_test)}")
print(f"GAN MAE test: {gan_mae_mean} +- {gan_mae_std}")

In [ ]:
linear_mse = mean_squared_error(ypred_linear_test, y_test)
gp_mse = mean_squared_error(ypred_gp_test, y_test)
gan_mse_mean = np.mean(np.asarray(mse_gan_))
gan_mse_std = np.std(np.asarray(mse_gan_))

print(f"Linear MSE test: {linear_mse}")
print(f"GP MSE test: {gp_mse}")
print(f"GAN MSE test: {gan_mse_mean} +- {gan_mse_std}")

In [ ]:
linear_nlpd = metrics.gaussian_NLPD(y_test, ypred_linear_test, np.ones(len(ypred_linear_test)), "Linear")

In [ ]:
gp_nlpd = metrics.gaussian_NLPD(y_test, ypred_gp_test, cov_test, "GP")

In [ ]:
gan_nlpd_train, w, lls = metrics.Parzen(cgan, X_valid, y_valid)
nlpd_ = []
for i in range(n_eval_runs):
    nlpd_.append(metrics.Parzen_test(cgan, X_test, y_test, w, exp_config.training.n_samples))
gan_nlpd_test = np.mean(nlpd_)
gan_nlpd_std_test = np.std(nlpd_)
print(f"GAN Train NLPD: {gan_nlpd_train}")
print(f"GAN Test NLPD: mean {gan_nlpd_test} std {gan_nlpd_std_test}")

In [ ]:
if exp_config.run.save_fig:
    file = open(f"{fig_dir}/metrics.txt","w") 

    file.write(f"===Test MAE===\n")
    file.write(f"Linear MAE test: {mean_absolute_error(ypred_linear_test, y_test)}\n") 
    file.write(f"GP MAE test: {mean_absolute_error(ypred_gp_test, y_test)}\n")
    file.write(f"GAN MAE test: {gan_mae_mean} +- {gan_mae_std}\n")
    file.write(f"===Test MSE===\n")
    file.write(f"Linear MSE test: {linear_mse}\n")
    file.write(f"GP MSE test: {gp_mse}\n")
    file.write(f"GAN MSE test: {gan_mse_mean} +- {gan_mse_std}\n")
    file.write(f"===Test NLPD===\n")
    file.write(f"Linear Gaussian NLPD: {linear_nlpd}\n")
    file.write(f"GP Gaussian NLPD: {gp_nlpd}\n")
    file.write(f"GAN NLPD: {gan_nlpd_test} +- {gan_nlpd_std_test}\n")
    file.close() 